# Implementation of MPI main-child paradigm

This template how to use MPI's main-child paradigm. As a use case we develop a basic sampler that provides the distribution of policy effects when there is uncertainty about the discount rate $\delta$ and the return to schooling $\beta_s$.

In [1]:
import sys
import os

import numpy as np

import respy as rp

import chaospy as cp


"""We create our samples here as a numpy array. This is the fastest alternative"""
distribution = cp.J(cp.Uniform(0.92, 0.98), cp.Uniform(0.03, 0.08))
sample = distribution.sample(10000, rule="random").T




In [2]:
params, options, df = rp.get_example_model("kw_94_one")


/home/peisenha/local/anaconda3/envs/ose-code-templates/lib/python3.7/site-packages/fastparquet/encoding.py:222: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy8 = numba.jitclass(spec8)(NumpyIO)
/home/peisenha/local/anaconda3/envs/ose-code-templates/lib/python3.7/site-packages/fastparquet/encoding.py:224: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy32 = numba.jitclass(spec32)(NumpyIO)
/home/pe

In [13]:
params.loc[["delta", ("wage_a", "exp_edu")], "value"]

/home/peisenha/local/anaconda3/envs/ose-code-templates/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


KeyError: "[('wage_a', 'exp_edu')] not in index"

In [26]:
params, options, df = rp.get_example_model("robinson_crusoe_basic")
simulate = rp.get_simulate_func(params, options)

In [27]:
df

Experience_Fishing  Shock_Reward_Fishing  \
Identifier Period                                             
0          0                        0             -0.035035   
           1                        0              0.074254   
           2                        0             -0.354560   
           3                        0             -0.109397   
           4                        0             -1.063705   
...                               ...                   ...   
999        0                        0              0.584099   
           1                        0             -0.391274   
           2                        0              0.394125   
           3                        0              0.531008   
           4                        0              1.367302   

                   Meas_Error_Wage_Fishing  Shock_Reward_Hammock  \
Identifier Period                                                  
0          0                             1              0.040965   
           1                             1              1.506491   
           2                             1              1.185316   
           3                             1             -0.785877   
           4                             1              1.245234   
...                                    ...                   ...   
999        0                             1              1.611990   
           1                             1              0.371305   
           2                             1             -1.448981   
           3                             1             -0.312350   
           4                             1              1.117095   

                   Meas_Error_Wage_Hammock  Dense_Key  Core_Index   Choice  \
Identifier Period                                                            
0          0                             1          0           0  hammock   
           1                             1          1           0  hammock   
           2                             1          2           0  hammock   
           3                             1          3           0  hammock   
           4                             1          4           0  hammock   
...                                    ...        ...         ...      ...   
999        0                             1          0           0  hammock   
           1                             1          1           0  hammock   
           2                             1          2           0  hammock   
           3                             1          3           0  hammock   
           4                             1          4           0  hammock   

                   Wage  Discount_Rate  ...  Nonpecuniary_Reward_Fishing  \
Identifier Period                       ...                                
0          0        NaN           0.95  ...                         -0.2   
           1        NaN           0.95  ...                         -0.2   
           2        NaN           0.95  ...                         -0.2   
           3        NaN           0.95  ...                         -0.2   
           4        NaN           0.95  ...                         -0.2   
...                 ...            ...  ...                          ...   
999        0        NaN           0.95  ...                         -0.2   
           1        NaN           0.95  ...                         -0.2   
           2        NaN           0.95  ...                         -0.2   
           3        NaN           0.95  ...                         -0.2   
           4        NaN           0.95  ...                         -0.2   

                   Wage_Fishing  Flow_Utility_Fishing  Value_Function_Fishing  \
Identifier Period                                                               
0          0           0.982635              0.782635                9.172605   
           1           1.037825              0.837825              

In [15]:
df = simulate(params)


In [19]:
df.groupby("Identifier")["Experience_Edu"].max().mean()

12.55

In [21]:
import pandas as pd

In [34]:
list_ = [[0.394, 0.9498696454239642, 0.05853213021033355], [0.381, 0.979098455534102, 0.030312000694920786]]
df = pd.DataFrame(list_, columns=["qoi", "delta", "exp_edu"])
df.index.name="sample"
df.dtypes

qoi        float64
delta      float64
exp_edu    float64
dtype: object

In [ ]:
sr

In [35]:
str_ = "subdir_child_1"

In [37]:
int(str_.split("_")[-1])

1

In [49]:
import glob

dfs = list()
for subdir in glob.glob("subdir_child_*"):
    rank = subdir.split("_")[-1]
    fname = f"{subdir}/rslt_child_{rank}.pkl"
    df = pd.read_pickle(fname)
    df["rank"] = rank
    dfs.append(df)

In [50]:
pd.concat(dfs, axis=0, ignore_index=True)

,qoi,delta,exp_edu,rank
0,0.393,0.956309,0.050480,0
1,0.393,0.925009,0.052838,0
2,0.403,0.973530,0.072859,1
3,0.401,0.949323,0.069179,1


In [46]:
df

,qoi,delta,exp_edu,rank
sample,,,,
0,0.403,0.973530,0.072859,1
1,0.401,0.949323,0.069179,1
